In [1]:
import os
import pandas as pd
import numpy as np
import datetime

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [10]:
engine = create_engine("sqlite:///battingStatsComplete.sqlite", echo=False)

In [3]:
standardBatting = pd.read_csv("standardBatting.csv")
advancedBatting = pd.read_csv("advancedBatting.csv")
valueBatting = pd.read_csv("valueBatting.csv")
teamsInfo = pd.read_csv("Teams.csv")
salariesInfo = pd.read_csv("Salaries.csv")
playerID = pd.read_csv("People.csv")


In [12]:
standardBatting.head()

,Name,Team,G,AB,PA,H,1B,2B,3B,HR,...,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,playerid
0,Ty Cobb,- - -,3035,11434,13072,4189,3053,724,295,117,...,NaN,357.0,94.0,NaN,295.0,NaN,892.0,178.0,0.366,1002378
1,Ross Barnes,- - -,499,2392,2506,859,665,144,44,6,...,NaN,65.0,NaN,NaN,NaN,NaN,73.0,25.0,0.359,1000573
2,Rogers Hornsby,- - -,2259,8173,9475,2930,1919,541,169,301,...,NaN,679.0,48.0,NaN,216.0,3.0,135.0,64.0,0.358,1006030
3,Joe Jackson,- - -,1332,4981,5690,1772,1243,307,168,54,...,NaN,158.0,59.0,NaN,131.0,NaN,202.0,61.0,0.356,1006301
4,Levi Meyerle,- - -,307,1443,1453,513,386,86,31,10,...,NaN,14.0,NaN,NaN,NaN,NaN,19.0,3.0,0.356,1008830


In [13]:
advancedBatting.head()

,Name,Team,PA,BB%,K%,BB/K,AVG,OBP,SLG,OPS,...,Spd,BABIP,UBR,wGDP,wSB,wRC,wRAA,wOBA,wRC+,playerid
0,Babe Ruth,- - -,10616,19.4 %,12.5 %,1.55,0.342,0.474,0.690,1.164,...,5.1,0.340,NaN,NaN,-23.4,2727,1437.2,0.513,197,1011327
1,Ted Williams,Red Sox,9791,20.6 %,7.2 %,2.85,0.344,0.482,0.634,1.116,...,3.5,0.328,NaN,NaN,-1.6,2349,1219.8,0.493,188,1014040
2,Lou Gehrig,Yankees,9660,15.6 %,8.2 %,1.91,0.340,0.447,0.632,1.080,...,5.5,0.332,NaN,NaN,-27.2,2265,1046.8,0.477,173,1004598
3,Jimmie Foxx,- - -,9670,15.0 %,13.6 %,1.11,0.325,0.428,0.609,1.038,...,5.1,0.336,NaN,NaN,-18.6,2136,934.2,0.460,158,1004285
4,Rogers Hornsby,- - -,9475,11.0 %,7.2 %,1.53,0.358,0.434,0.577,1.010,...,5.6,0.365,NaN,NaN,-1.8,2018,899.0,0.459,173,1006030


In [14]:
valueBatting.head()

,Name,Team,Batting,Base Running,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars,playerid
0,Babe Ruth,- - -,1370.8,-23.4,79.0,-97.6,1347.3,-18.6,47.9,325.2,1701.9,168.4,NaN,1011327
1,Barry Bonds,- - -,1143.4,30.4,189.6,-122.0,1173.8,67.6,28.1,384.4,1653.9,164.4,$164.7,1109
2,Willie Mays,- - -,804.6,32.9,185.0,-14.9,837.5,170.1,42.1,357.5,1407.2,149.9,NaN,1008315
3,Ty Cobb,- - -,975.4,60.6,0.0,-90.0,1036.0,-90.0,47.9,377.1,1371.0,149.3,NaN,1002378
4,Honus Wagner,- - -,647.8,56.9,85.0,99.4,704.7,184.4,50.0,346.5,1285.6,138.1,NaN,1013485


In [15]:
standardBatting_DF = pd.DataFrame(standardBatting)
advancedBatting_DF = pd.DataFrame(advancedBatting)
valueBatting_DF = pd.DataFrame(valueBatting)

In [18]:
first_merge = pd.merge(standardBatting_DF, advancedBatting_DF, on="Name")
first_merge.head()

,Name,Team_x,G,AB,PA_x,H,1B,2B,3B,HR,...,Spd,BABIP,UBR,wGDP,wSB,wRC,wRAA,wOBA,wRC+,playerid_y
0,Ty Cobb,- - -,3035,11434,13072,4189,3053,724,295,117,...,7.5,0.378,NaN,NaN,60.6,2534,1093.8,0.445,165,1002378
1,Ross Barnes,- - -,499,2392,2506,859,665,144,44,6,...,7.4,0.368,NaN,NaN,0.6,658,231.9,0.388,148,1000573
2,Rogers Hornsby,- - -,2259,8173,9475,2930,1919,541,169,301,...,5.6,0.365,NaN,NaN,-1.8,2018,899.0,0.459,173,1006030
3,Joe Jackson,- - -,1332,4981,5690,1772,1243,307,168,54,...,6.5,0.351,NaN,NaN,-10.6,1090,480.3,0.443,165,1006301
4,Levi Meyerle,- - -,307,1443,1453,513,386,86,31,10,...,7.0,0.357,NaN,NaN,4.1,381,119.9,0.376,139,1008830


In [19]:
second_merge = pd.merge(first_merge, valueBatting_DF, on="Name")
second_merge.head()

,Name,Team_x,G,AB,PA_x,H,1B,2B,3B,HR,...,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars,playerid
0,Ty Cobb,- - -,3035,11434,13072,4189,3053,724,295,117,...,0.0,-90.0,1036.0,-90.0,47.9,377.1,1371.0,149.3,NaN,1002378
1,Ross Barnes,- - -,499,2392,2506,859,665,144,44,6,...,57.0,15.4,221.0,72.4,9.8,96.9,400.0,30.4,NaN,1000573
2,Rogers Hornsby,- - -,2259,8173,9475,2930,1919,541,169,301,...,54.0,72.5,862.1,126.5,23.5,284.8,1296.9,130.3,NaN,1006030
3,Joe Jackson,- - -,1332,4981,5690,1772,1243,307,168,54,...,11.0,-60.1,418.6,-49.1,24.4,161.1,555.0,60.5,NaN,1006301
4,Levi Meyerle,- - -,307,1443,1453,513,386,86,31,10,...,-24.0,5.3,111.7,-18.7,6.0,58.4,157.4,10.1,NaN,1008830


In [25]:
second_merge.columns

Index(['Name', 'Team_x', 'G', 'AB', 'PA_x', 'H', '1B', '2B', '3B', 'HR', 'R',
       'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'AVG_x',
       'playerid_x', 'Team_y', 'PA_y', 'BB%', 'K%', 'BB/K', 'AVG_y', 'OBP',
       'SLG', 'OPS', 'ISO', 'Spd', 'BABIP', 'UBR', 'wGDP', 'wSB', 'wRC',
       'wRAA', 'wOBA', 'wRC+', 'playerid_y', 'Team', 'Batting', 'Base Running',
       'Fielding', 'Positional', 'Offense', 'Defense', 'League', 'Replacement',
       'RAR', 'WAR', 'Dollars', 'playerid'],
      dtype='object')

In [35]:
second_merge_reduced = second_merge.loc[:,['Name', 'Team_x', 'G', 'AB', 'PA_x', 'H', '1B', '2B', '3B', 'HR', 'R',
       'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'AVG_x','BB%', 'K%', 'BB/K', 'OBP',
       'SLG', 'OPS', 'ISO', 'Spd', 'BABIP', 'UBR', 'wGDP', 'wSB', 'wRC','wRAA', 'wOBA', 'wRC+', 'Team', 'Batting', 
       'Base Running', 'Fielding', 'Positional', 'Offense', 'Defense', 'League', 'Replacement', 'RAR', 'WAR', 
       'Dollars', 'playerid']]

second_merge_reduced.sample(10)

,Name,Team_x,G,AB,PA_x,H,1B,2B,3B,HR,...,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars,playerid
1362,Chris Shelton,- - -,299,930,1044,254,164,46,7,37,...,4.2,-19.6,13.2,-15.4,3.5,32.2,33.5,3.3,$16.4,1891
3879,Fred Smith,- - -,438,1422,1607,321,249,39,25,8,...,-13.0,17.2,-39.7,4.2,3.1,44.8,12.4,1.6,NaN,1012141
2204,Chris Gwynn,- - -,599,1007,1100,263,199,36,11,17,...,-12.0,-9.0,-22.1,-21.0,2.6,33.0,-7.4,-0.7,NaN,1005165
3790,Dick Schofield,- - -,1368,4299,4928,989,764,137,32,56,...,19.0,40.0,-99.7,59.0,13.5,99.8,72.6,7.9,NaN,1011601
2323,Bill Rigney,Giants,654,1966,2214,510,377,78,14,41,...,-6.0,19.4,-20.1,13.4,5.3,65.3,64.0,6.6,NaN,1010960
1733,Russ Nixon,- - -,906,2504,2714,670,509,115,19,27,...,-36.0,37.8,-65.6,1.8,-0.6,77.7,13.4,1.6,NaN,1009609
3443,Lennie Merullo,Cubs,639,2071,2264,497,391,92,8,6,...,-13.0,37.1,-75.9,24.1,3.8,63.6,15.6,1.9,NaN,1008796
257,Ben Chapman,- - -,1717,6478,7420,1958,1354,407,107,90,...,50.0,-43.2,151.9,6.8,39.4,231.8,429.9,41.3,NaN,1002141
2100,Steve O'Neill,- - -,1590,4795,5560,1259,964,248,34,13,...,11.0,47.3,-63.9,58.3,21.8,161.2,177.4,18.4,NaN,1009820
2396,Lou Johnson,- - -,676,2049,2251,529,370,97,14,48,...,8.0,-27.5,9.8,-19.5,8.6,61.9,60.7,6.8,NaN,1006484


In [28]:
second_merge_reduced.columns

Index(['Name', 'Team_x', 'G', 'AB', 'PA_x', 'H', '1B', '2B', '3B', 'HR', 'R',
       'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'AVG_x',
       'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'OPS', 'ISO', 'Spd', 'BABIP', 'UBR',
       'wGDP', 'wSB', 'wRC', 'wRAA', 'wOBA', 'wRC+', 'Team', 'Batting',
       'Base Running', 'Fielding', 'Positional', 'Offense', 'Defense',
       'League', 'Replacement', 'RAR', 'WAR', 'Dollars', 'playerid'],
      dtype='object')

In [29]:
second_merge_reduced.rename(columns={"Team_x": "Team", "PA_x": "PA", "AVG_x": "AVG"}, inplace=True)
second_merge_reduced

,Name,Team,G,AB,PA,H,1B,2B,3B,HR,...,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars,playerid
0,Ty Cobb,- - -,3035,11434,13072,4189,3053,724,295,117,...,0.0,-90.0,1036.0,-90.0,47.9,377.1,1371.0,149.3,NaN,1002378
1,Ross Barnes,- - -,499,2392,2506,859,665,144,44,6,...,57.0,15.4,221.0,72.4,9.8,96.9,400.0,30.4,NaN,1000573
2,Rogers Hornsby,- - -,2259,8173,9475,2930,1919,541,169,301,...,54.0,72.5,862.1,126.5,23.5,284.8,1296.9,130.3,NaN,1006030
3,Joe Jackson,- - -,1332,4981,5690,1772,1243,307,168,54,...,11.0,-60.1,418.6,-49.1,24.4,161.1,555.0,60.5,NaN,1006301
4,Levi Meyerle,- - -,307,1443,1453,513,386,86,31,10,...,-24.0,5.3,111.7,-18.7,6.0,58.4,157.4,10.1,NaN,1008830
5,Lefty O'Doul,- - -,970,3264,3659,1140,811,175,41,113,...,-15.0,-34.7,217.2,-49.7,4.8,115.8,288.2,27.0,NaN,1009734
6,Ed Delahanty,- - -,1835,7505,8389,2596,1788,522,185,101,...,20.0,-73.4,574.2,-53.4,46.1,297.0,863.9,73.7,NaN,1003155
7,Cal McVey,- - -,530,2513,2543,869,682,132,44,11,...,-5.0,1.9,171.5,-3.1,8.2,96.9,273.5,20.6,NaN,1008696
8,Tris Speaker,- - -,2789,10195,11988,3514,2383,792,222,117,...,92.0,-67.6,815.2,24.4,46.2,349.0,1234.8,130.6,NaN,1012309
9,Ted Williams,Red Sox,2292,7706,9791,2654,1537,525,71,521,...,-29.0,-96.1,1064.5,-125.1,34.2,289.7,1263.2,130.4,NaN,1014040


In [45]:
playerID.columns

Index(['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry',
       'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay',
       'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast',
       'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame',
       'retroID', 'bbrefID'],
      dtype='object')

In [31]:
teamsInfo.columns

Index(['yearID', 'lgID', 'teamID', 'franchID', 'divID', 'Rank', 'G', 'Ghome',
       'W', 'L', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA',
       'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP',
       'name', 'park', 'attendance', 'BPF', 'PPF', 'teamIDBR',
       'teamIDlahman45', 'teamIDretro'],
      dtype='object')

In [50]:
salariesInfo.head(10)

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000
5,1985,ATL,NL,chambch01,800000
6,1985,ATL,NL,dedmoje01,150000
7,1985,ATL,NL,forstte01,483333
8,1985,ATL,NL,garbege01,772000
9,1985,ATL,NL,harpete01,250000


In [42]:
playerID_DF = pd.DataFrame(playerID)
teamsInfo_DF = pd.DataFrame(teamsInfo)
salariesInfo_DF = pd.DataFrame(salariesInfo)

In [83]:
salariesInfo_DF.groupby(["playerID", "teamID"], as_index=False).count()

,playerID,teamID,yearID,lgID,salary
0,aardsda01,BOS,1,1,1
1,aardsda01,CHA,1,1,1
2,aardsda01,NYA,1,1,1
3,aardsda01,SEA,3,3,3
4,aardsda01,SFN,1,1,1
5,aasedo01,BAL,3,3,3
6,aasedo01,NYN,1,1,1
7,abadan01,CIN,1,1,1
8,abadfe01,HOU,2,2,2
9,abadfe01,MIN,1,1,1


In [88]:
third_merge = pd.merge(salariesInfo_DF, playerID_DF, on="playerID", how="left")
third_merge.head()

,yearID,teamID,lgID,playerID,salary,birthYear,birthMonth,birthDay,birthCountry,birthState,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,1985,ATL,NL,barkele01,870000,1955.0,7.0,7.0,USA,KY,...,Barker,Leonard Harold,225.0,77.0,R,R,1976-09-14,1987-09-26,barkl001,barkele01
1,1985,ATL,NL,bedrost01,550000,1957.0,12.0,6.0,USA,MA,...,Bedrosian,Stephen Wayne,200.0,75.0,R,R,1981-08-14,1995-08-09,bedrs001,bedrost01
2,1985,ATL,NL,benedbr01,545000,1955.0,8.0,18.0,USA,AL,...,Benedict,Bruce Edwin,175.0,73.0,R,R,1978-08-18,1989-09-11,beneb001,benedbr01
3,1985,ATL,NL,campri01,633333,1953.0,6.0,10.0,USA,GA,...,Camp,Rick Lamar,195.0,73.0,R,R,1976-09-15,1985-10-05,campr001,campri01
4,1985,ATL,NL,ceronri01,625000,1954.0,5.0,19.0,USA,NJ,...,Cerone,Richard Aldo,192.0,71.0,R,R,1975-08-17,1992-07-10,ceror001,ceronri01


In [90]:
third_merge.columns

Index(['yearID', 'teamID', 'lgID', 'playerID', 'salary', 'birthYear',
       'birthMonth', 'birthDay', 'birthCountry', 'birthState', 'birthCity',
       'deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathState',
       'deathCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height',
       'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID'],
      dtype='object')

In [49]:
salariesInfo_DF.shape

(26428, 5)

In [92]:
teamsInfo_DF.columns

Index(['yearID', 'lgID', 'teamID', 'franchID', 'divID', 'Rank', 'G', 'Ghome',
       'W', 'L', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA',
       'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP',
       'name', 'park', 'attendance', 'BPF', 'PPF', 'teamIDBR',
       'teamIDlahman45', 'teamIDretro'],
      dtype='object')

In [93]:
fourth_merge = pd.merge(third_merge, teamsInfo_DF, on=["yearID", "teamID"], how="left")
fourth_merge.head()

,yearID,teamID,lgID_x,playerID,salary,birthYear,birthMonth,birthDay,birthCountry,birthState,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1985,ATL,NL,barkele01,870000,1955.0,7.0,7.0,USA,KY,...,197,0.976,Atlanta Braves,Atlanta-Fulton County Stadium,1350137.0,105,106,ATL,ATL,ATL
1,1985,ATL,NL,bedrost01,550000,1957.0,12.0,6.0,USA,MA,...,197,0.976,Atlanta Braves,Atlanta-Fulton County Stadium,1350137.0,105,106,ATL,ATL,ATL
2,1985,ATL,NL,benedbr01,545000,1955.0,8.0,18.0,USA,AL,...,197,0.976,Atlanta Braves,Atlanta-Fulton County Stadium,1350137.0,105,106,ATL,ATL,ATL
3,1985,ATL,NL,campri01,633333,1953.0,6.0,10.0,USA,GA,...,197,0.976,Atlanta Braves,Atlanta-Fulton County Stadium,1350137.0,105,106,ATL,ATL,ATL
4,1985,ATL,NL,ceronri01,625000,1954.0,5.0,19.0,USA,NJ,...,197,0.976,Atlanta Braves,Atlanta-Fulton County Stadium,1350137.0,105,106,ATL,ATL,ATL


In [94]:
fourth_merge.columns

Index(['yearID', 'teamID', 'lgID_x', 'playerID', 'salary', 'birthYear',
       'birthMonth', 'birthDay', 'birthCountry', 'birthState', 'birthCity',
       'deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathState',
       'deathCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height',
       'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID', 'lgID_y',
       'franchID', 'divID', 'Rank', 'G', 'Ghome', 'W', 'L', 'DivWin', 'WCWin',
       'LgWin', 'WSWin', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB',
       'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA', 'CG', 'SHO', 'SV', 'IPouts', 'HA',
       'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP', 'name', 'park', 'attendance',
       'BPF', 'PPF', 'teamIDBR', 'teamIDlahman45', 'teamIDretro'],
      dtype='object')

In [95]:
fourth_merge_reduced = fourth_merge.loc[:,['yearID', 'teamID', 'lgID_x', 'playerID', 'salary', 'birthYear',
       'birthMonth', 'birthDay', 'birthCountry', 'birthState', 'birthCity',
       'deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathState',
       'deathCity', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height',
       'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID',
       'franchID', 'divID', 'Rank', 'G', 'Ghome', 'W', 'L', 'DivWin', 'WCWin',
       'LgWin', 'WSWin', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB',
       'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA', 'CG', 'SHO', 'SV', 'IPouts', 'HA',
       'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP', 'name', 'park', 'attendance',
       'BPF', 'PPF']]

In [96]:
fourth_merge_reduced.rename(columns={"lgID_x": "lgID"}, inplace=True)
fourth_merge_reduced.head()

,yearID,teamID,lgID,playerID,salary,birthYear,birthMonth,birthDay,birthCountry,birthState,...,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF
0,1985,ATL,NL,barkele01,870000,1955.0,7.0,7.0,USA,KY,...,642,776,159,197,0.976,Atlanta Braves,Atlanta-Fulton County Stadium,1350137.0,105,106
1,1985,ATL,NL,bedrost01,550000,1957.0,12.0,6.0,USA,MA,...,642,776,159,197,0.976,Atlanta Braves,Atlanta-Fulton County Stadium,1350137.0,105,106
2,1985,ATL,NL,benedbr01,545000,1955.0,8.0,18.0,USA,AL,...,642,776,159,197,0.976,Atlanta Braves,Atlanta-Fulton County Stadium,1350137.0,105,106
3,1985,ATL,NL,campri01,633333,1953.0,6.0,10.0,USA,GA,...,642,776,159,197,0.976,Atlanta Braves,Atlanta-Fulton County Stadium,1350137.0,105,106
4,1985,ATL,NL,ceronri01,625000,1954.0,5.0,19.0,USA,NJ,...,642,776,159,197,0.976,Atlanta Braves,Atlanta-Fulton County Stadium,1350137.0,105,106


In [11]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

[]

In [ ]:
inspectorinspect  = inspect(engine)
inspector.get_table_names()